In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import json
with open("../../../api_keys.json", mode = 'r') as f:
    api_keys = json.load(f)

In [4]:
import os

# os.environ["GOOGLE_API_KEY"] = api_keys['ashishjain1547'] 
# os.environ["MODEL"] = "gemini/gemini-2.0-flash"

os.environ['OPENAI_API_KEY'] = api_keys['openai']
os.environ['MODEL'] = 'gpt-3.5-turbo'

In [5]:
# import litellm

# # litellm.api_key = api_keys['ashishjain1547']  # Ensure this is set
# # litellm.model = "gemini/gemini-2.0-flash"  # Ensure Gemini is being used

In [6]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class Score(BaseModel):
    score: int

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

In [7]:
grammar_expert = Agent(
    role="Grammar Expert",
    goal="Evaluate a Hindi-English sentence pair for grammatical correctness."
    "Hindi: {hindi}\nEnglish: {english}",
    backstory="You're working on a language learning app for novice English learner who are fluent in Hindi."
              "You evaluate sentences for grammatical correctness."
              "You return your answer as 0 if sentence pair are grammatically incorrect and 1 if sentence pair are grammatically correct."
              "You give 0 also in the case if either English or Hindi sentence alone is grammatically incorrect.",
    allow_delegation=False,
	verbose=True
)

In [8]:
profanity_checker = Agent(
    role="Profanity Checker",
    goal="Evaluate a Hindi-English sentence pair for profanity"
    "Hindi: {hindi}\nEnglish: {english}",
    backstory="You're working on a language learning app for novice English learner who are fluent in Hindi."
    "You evaluate sentences for profanity."
    "If the sentence (or both sentences) are double meaning, directly or indirectly vulgar, or contain profanity, you return 0."
    "Otherwise, you return 1.",
    allow_delegation=False,
    verbose=True
)

In [9]:
non_veg_food_checker = Agent(
    role="Non-veg food marker",
    goal="Evaluate a Hindi-English sentence pair if it contains anything about non-veg food"
    "Hindi: {hindi}\nEnglish: {english}",
    backstory="You're working on a language learning app for novice English learner who are fluent in Hindi."
    "You evaluate sentences for profanity."
    "If the sentence (or both sentences) are about non-veg food, you return 0."
    "Otherwise if sentence is not about non-veg food, you return 1.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

In [10]:
check_grammar = Task(
    description=(
        "1. Check the grammar of the Hindi sentence.\n"
        "2. Check the grammar of the English sentence.\n"
        "3. Check that Hindi-English sentence pair mean the same thing."
    ),
    expected_output="Numeric Score. 0 if grammar is incorrect, and 1 if grammar is correct",
    output_json=Score,
    agent=grammar_expert
)

In [11]:
profanity_check = Task(
    description=(
        "Check the Hindi-English sentence pair for profanity, vulgarness or double meaning."
    ),
    expected_output="Numeric Score. 0 if profanity is there, and 1 if profanity is NOT there",
    output_json=Score,
    agent=profanity_checker
)

In [12]:
non_veg_food_check = Task(
    description=(
        "1. Check the Hindi sentence for presence of non-veg food.\n"
        "2. Check the English sentence for presence of non-veg food."
    ),
    expected_output="Numeric Score. 0 if non-veg is there, and 1 if sentences are NOT about non-veg food",
    output_json=Score,
    agent=non_veg_food_checker
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=True` allows you to see all the logs of the execution. 

In [13]:
crew = Crew(
    agents=[grammar_expert, profanity_checker, non_veg_food_checker],
    tasks=[check_grammar, profanity_check, non_veg_food_check],
    verbose=True
)

In [15]:
result = crew.kickoff(inputs={"hindi": "मैं दूध हूँ", "english": "I am milk"})

for task in result.tasks_output:
    print(f"Agent: {task.agent}")
    print(f"Description: {task.description}")
    print(f"Expected Output: {task.expected_output}")
    print(f"Task Result: {task.raw}\n")

 
[2025-03-13 21:57:19][🚀 CREW 'CREW' STARTED, FD4E7432-5049-4202-B6BC-426EE48E31BD]: 2025-03-13 21:57:19.237629
 
[2025-03-13 21:57:19][📋 TASK STARTED: 1. CHECK THE GRAMMAR OF THE HINDI SENTENCE.
2. CHECK THE GRAMMAR OF THE ENGLISH SENTENCE.
3. CHECK THAT HINDI-ENGLISH SENTENCE PAIR MEAN THE SAME THING.]: 2025-03-13 21:57:19.259061
 
[2025-03-13 21:57:19][🤖 AGENT 'GRAMMAR EXPERT' STARTED TASK]: 2025-03-13 21:57:19.260204
# Agent: Grammar Expert
## Task: 1. Check the grammar of the Hindi sentence.
2. Check the grammar of the English sentence.
3. Check that Hindi-English sentence pair mean the same thing.
 
[2025-03-13 21:57:19][🤖 LLM CALL STARTED]: 2025-03-13 21:57:19.260388
 
[2025-03-13 21:57:19][✅ LLM CALL COMPLETED]: 2025-03-13 21:57:19.816198
 
[2025-03-13 21:57:19][🤖 LLM CALL STARTED]: 2025-03-13 21:57:19.818292
 
[2025-03-13 21:57:20][✅ LLM CALL COMPLETED]: 2025-03-13 21:57:20.686973


# Agent: Grammar Expert
## Final Answer: 
{
  "score": 0
}


 
[2025-03-13 21:57:20][✅ AGENT '

## Running the Crew

In [16]:
# # Agent: Hindi-English Grammar Expert
# # Description: 1. Check the grammar of the Hindi sentence.
# # 2. Check the grammar of the English sentence.
# # 3. Check that Hindi-English sentence pair mean the same thing.
# # Expected Output: 0 if grammar is incorrect, and 1 if grammar is correct
# # Task Result: 0

# # Agent: Profanity Checker
# # Description: 1. Check the Hindi sentence for profanity, vulgarness or double meaning.
# # 2. Check the English sentence for profanity, vulgarness or double meaning.
# # Expected Output: 0 if profanity is there, and 1 if profanity is NOT there
# # Task Result: 1

In [17]:
import pandas as pd

In [18]:
df_in = pd.read_excel("Sentences - Itr 4 - 2025 Feb 28.xlsx")

In [19]:
df5 = df_in.iloc[:5]

In [20]:
df5

,word,object,hindi,english,pos,len,source
0,I,kiwi,मैं कीवी खाता हूँ।,I eat kiwi.,verb,3,ITR 2
1,I,owl,मुझे उल्लू पसंद हैं।,I like owls.,verb,3,ITR 2
2,I,cookies,मैं कुकीज़ बनाता हूँ।,I bake cookies.,verb,3,ITR 2
3,I,cookies,मुझे कुकीज़ चाहिए।,I want cookies.,pronoun,3,ITR 2
4,I,cookies,मुझे कुकीज़ पसंद हैं।,I like cookies.,pronoun,3,ITR 2


In [27]:
def get_scores(row):
    result = crew.kickoff(inputs={"hindi": row['hindi'], "english": row['english']})
    
    grammar, profanity, nonveg = None, None, None
    for task in result.tasks_output:
        print(task.agent)
        print(task.raw)
        if task.agent == "Grammar Expert":
            grammar = json.loads(task.raw)['score'] 

        elif task.agent == "Profanity Checker":
            profanity = json.loads(task.raw)['score']

        elif task.agent == "Non-veg food marker":
            nonveg = json.loads(task.raw)['score']
        
    return pd.Series([grammar, profanity, nonveg], index=['grammar', 'profanity', 'nonveg'])

# Apply function to each row
df5[['grammar', 'profanity', 'nonveg']] = df5.apply(get_scores, axis=1)

 
[2025-03-13 22:03:37][🚀 CREW 'CREW' STARTED, FD4E7432-5049-4202-B6BC-426EE48E31BD]: 2025-03-13 22:03:37.627168
 
[2025-03-13 22:03:37][📋 TASK STARTED: 1. CHECK THE GRAMMAR OF THE HINDI SENTENCE.
2. CHECK THE GRAMMAR OF THE ENGLISH SENTENCE.
3. CHECK THAT HINDI-ENGLISH SENTENCE PAIR MEAN THE SAME THING.]: 2025-03-13 22:03:37.651386
 
[2025-03-13 22:03:37][🤖 AGENT 'GRAMMAR EXPERT' STARTED TASK]: 2025-03-13 22:03:37.653552
# Agent: Grammar Expert
## Task: 1. Check the grammar of the Hindi sentence.
2. Check the grammar of the English sentence.
3. Check that Hindi-English sentence pair mean the same thing.
 
[2025-03-13 22:03:37][🤖 LLM CALL STARTED]: 2025-03-13 22:03:37.653892
 
[2025-03-13 22:03:38][✅ LLM CALL COMPLETED]: 2025-03-13 22:03:38.519187


# Agent: Grammar Expert
## Final Answer: 
{
  "score": 1
}


 
[2025-03-13 22:03:38][✅ AGENT 'GRAMMAR EXPERT' COMPLETED TASK]: 2025-03-13 22:03:38.519408
 
[2025-03-13 22:03:38][✅ TASK COMPLETED: 1. CHECK THE GRAMMAR OF THE HINDI SENTENCE.


In [28]:
df5

,word,object,hindi,english,pos,len,source,grammar,profanity,nonveg
0,I,kiwi,मैं कीवी खाता हूँ।,I eat kiwi.,verb,3,ITR 2,1,1,1
1,I,owl,मुझे उल्लू पसंद हैं।,I like owls.,verb,3,ITR 2,1,1,1
2,I,cookies,मैं कुकीज़ बनाता हूँ।,I bake cookies.,verb,3,ITR 2,1,1,1
3,I,cookies,मुझे कुकीज़ चाहिए।,I want cookies.,pronoun,3,ITR 2,1,1,1
4,I,cookies,मुझे कुकीज़ पसंद हैं।,I like cookies.,pronoun,3,ITR 2,1,1,1
